In [61]:
%matplotlib inline
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import logm

In [62]:
## parameters, UD=underdamped, OD=overdamped
N   = 2**20            # # of data points
m   = 1                # mass
T   = 1                # temperature
k   = 1                # stiffness
k_B = 1                # Boltzmann constant
f_s = 1                # sampling frequency
dt  = 1/f_s            # step size
g_c = np.sqrt(4*m*k)   # gamma critical
g_f = .5               # <1 for UD and >1 for OD
g   = g_c*g_f          # gamma
w0  = np.sqrt(k/m)     # frequency of UD motion
tau = m/g              # relaxation time
D   = k_B*T/g          # diffusion constant

if g_f >1:
    w = 1j*np.sqrt(-(w0**2)+(1/(4*tau*tau))) 
else:
    w = np.sqrt((w0**2)-(1/(4*tau*tau)))

params = (N, dt, w, w0, tau, D, k, m, k_B, T)

In [63]:
def calcSigma(params):
    N, dt, w, w0, tau, D, k, m, k_B, T = params
    kmw=k/(m*w*w);   dtbt=-dt/tau; ee=np.exp(dtbt);  dd=D/(w*w*tau*tau);  
    tt   = w*dt;   cc  = np.cos(tt);  ss=np.sin(tt)
    
    s1 = (k_B*T/k)*(1-ee*(kmw*ss*ss+(cc+ss/(2*w*tau))**2))
    s2 = dd*ee*ss*ss
    s3 = (k_B*T/m)*(1-ee*(kmw*ss*ss+(cc-ss/(2*w*tau))**2)) 
    return np.real(s1), np.real(s3), np.real(s2) 


def calcLambda(params):
    N, dt, w, w0, tau, D, k, m, k_B, T = params
    ii  = np.eye(2)
    ll = np.asanyarray([[0, -1], [k/m, g/m]])
    ee = np.exp(-dt/(2*tau)); wt2=2*w*tau
    cc=np.cos(w*dt); ss=np.sin(w*dt) 
    
    Lambda = ee*((cc+ss/wt2)*ii - ll*ss/w ) 
    return np.real(Lambda)


def calcXV(params, Lambda, Sigma):
    N, dt, w, w0, tau, D, k, m, k_B, T = params
    x = np.zeros([N,1])
    v = np.zeros([N,1])
    
    s1, s3, s2 = Sigma

    for j in np.arange(0,N-1):
        oldvec = np.array([x[j],v[j]])
        randgauss = np.random.randn(2,1)
        delx = np.sqrt(s1)*randgauss[0]
        delv = (s2/(np.sqrt(s1)))*randgauss[0]+(np.sqrt(s3 - ((s2**2)/(s1))))*randgauss[1]
        delvec = np.array([delx,delv])
        updatevec = np.dot(Lambda,oldvec)+delvec
        x[j+1] = updatevec[0]
        v[j+1] = updatevec[1]
    return x,v

In [64]:
params = (N, dt, w, w0, tau, D, k, m, k_B, T)
Lambda = calcLambda(params)
Sigma  = calcSigma(params)
x, v   = calcXV(params, Lambda, Sigma)

In [65]:
#plt.plot(x);plt.plot(v);

## Bayes I

In [66]:
# matrix sufficient statistics
T1_11 = np.sum(x[1:]**2)
T1_12 = np.sum(x[1:]*v[1:])
T1_21 = T1_12
T1_22 = np.sum(v[1:]**2)

T2_11 = np.sum(x[1:]*x[:-1])
T2_12 = np.sum(x[1:]*v[:-1])
T2_21 = np.sum(v[1:]*x[:-1])
T2_22 = np.sum(v[1:]*v[:-1])

T3_11 = np.sum(x[:-1]*x[:-1])
T3_12 = np.sum(x[:-1]*v[:-1])
T3_21 = T3_12
T3_22 = np.sum(v[:-1]*v[:-1])

T1    = np.asanyarray([[T1_11, T1_12],[T1_21, T1_22]])
T2    = np.asanyarray([[T2_11, T2_12],[T2_21, T2_22]])
T3    = np.asanyarray([[T3_11, T3_12],[T3_21, T3_22]])

In [67]:
#MAP estimate of Lambda and Sigma

invT3     = np.linalg.inv(T3)
LambdaMAP = np.dot(T2, invT3)
SigmaMAP  = (1/N)*( T1 - np.dot( T2, np.dot(invT3, T2.transpose()) ) )

In [68]:
#MAP estimate of c using Onsager-Casimir symmetry

L_t  = LambdaMAP.transpose()
eps  = np.asanyarray([[1,0],[0,-1]])

L_te = np.dot(eps, L_t)
Lte2 = np.dot(L_te, L_te)

II   = np.eye(2)
cc   = np.linalg.inv(II - Lte2)
cMAP = np.dot(SigmaMAP, cc)

print LambdaMAP, '\n', SigmaMAP, '\n', cMAP

[[ 0.65952885  0.53362085]
 [-0.53373082  0.12652342]] 
[[ 0.28036001  0.28503431]
 [ 0.28503431  0.70042811]] 
[[  9.99533690e-01   9.80490846e-04]
 [  6.35690929e-04   1.00152369e+00]]


In [69]:
#MAP estimate of k and m and gamma

ll = -logm(LambdaMAP)/dt

m_MAP = k_B*T/cMAP[1,1]
k_MAP = k_B*T/cMAP[0,0]
g_MAP = m*ll[1,1]


print m_MAP, k_MAP, g_MAP
print 
print m, k, g, k_B*T/k, k_B*T/k

0.998478630186 1.0004665271 0.998803148808

1 1 1.0 1.0 1.0


## Bayes II

In [70]:
#MAP estimate of c using Bayes II
c_B2 = T3/N

k_B2 = k_B*T/c_B2[0,0]
m_B2 = k_B*T/c_B2[1,1]

print c_B2, '\n', k_B2, m_B2

[[  1.00121542e+00   1.85599362e-04]
 [  1.85599362e-04   1.00165247e+00]] 
0.998786056284 0.998350252413
